# The important parts of the code

## Command imports

In [359]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
import glob
import random
from scipy.ndimage import label
from numpy.linalg import inv

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## Import mse IDs 

In [360]:
#subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
subjects = ["mse1474"]
all_sub_results = [["mseID",
                   "lesion number and type",      
                   "number of subcortical lesions",
                   "number of juxtacortical lesions",
                   "number of periventricular lesions",
                   "number of infratentorial lesions",
                   "number of errors"]]
print subjects[:5]

['mse1474']


## Set variables to lesion/segmentation files, freeview cmdline copy and paste

In [361]:
for les_file in glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % subjects[0]):
    print les_file
for seg_file in glob.glob("/data/henry7/PBR/subjects/%s/masks/*/segmentation.nii.gz" % subjects[0]):
    print seg_file
for gm_file in glob.glob("/data/henry6/PBR/surfaces/*%s*/mri/ribbon.mgz" % subjects[0]):
    print gm_file

print
print "Freeview link:"
print 
print "freeview", seg_file, les_file, gm_file

/data/henry7/PBR/subjects/mse1474/lesions_manual/ms242-mse1474-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz
/data/henry7/PBR/subjects/mse1474/masks/ms242-mse1474-002-AX_T1_3D_IRSPGR/segmentation.nii.gz
/data/henry6/PBR/surfaces/ms242-mse1474-002-AX_T1_3D_IRSPGR/mri/ribbon.mgz

Freeview link:

freeview /data/henry7/PBR/subjects/mse1474/masks/ms242-mse1474-002-AX_T1_3D_IRSPGR/segmentation.nii.gz /data/henry7/PBR/subjects/mse1474/lesions_manual/ms242-mse1474-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz /data/henry6/PBR/surfaces/ms242-mse1474-002-AX_T1_3D_IRSPGR/mri/ribbon.mgz


## Binarize

enlarged_gm = Binarize(in_file=rib_file, match=3, dilate=3, binary_file='/home/mkhan/Temp_Binary/%s' % subjects[0])
enlarged_gm.cmdline

## Obtain affines 

In [362]:
les_img = nib.load(les_file)
les_img.dataobj
seg_img = nib.load(seg_file)
seg_img.dataobj
gm_img = nib.load(gm_file)
gm_img.dataobj

les_data = les_img.get_data()
print les_data.shape
seg_data = seg_img.get_data()
print seg_data.shape
gm_data = gm_img.get_data()
print gm_data.shape

les_affine = les_img.get_affine()
print les_affine
gm_affine = gm_img.get_affine()
print gm_affine
seg_affine = seg_img.get_affine()
print seg_affine
inv_seg_affine = np.linalg.inv(seg_affine)
print inv_seg_affine

(256, 256, 150)
(256, 256, 256)
(256, 256, 256)
[[  -0.9375        0.            0.          122.45800018]
 [   0.            0.9375        0.          -78.72650146]
 [   0.            0.            1.          -49.06119919]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          130.45800018]
 [   0.            0.            1.          -86.72650146]
 [   0.           -1.            0.          158.93880081]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          130.45800781]
 [   0.            0.            1.          -86.72650146]
 [   0.           -1.            0.          158.938797  ]
 [   0.            0.            0.            1.        ]]
[[  -1.           -0.           -0.          130.45800781]
 [  -0.           -0.           -1.          158.938797  ]
 [   0.            1.            0.           86.72650146]
 [   0.            0.            0.            1.        ]]


## set labels for coordinates

In [363]:
#lesion labels
les_labels, n_les_labels = label(les_data==[1])

#segmentation - brainstem labels
seg_brainstem_labels, n_seg_brainstem_labels = label(seg_data==[16])
seg_lcerebellumcortex_labels, n_seg_lcerebellumcortex_labels = label(seg_data==[8])
seg_rcerebellumcortex_labels, n_seg_rcerebellumcortex_labels = label(seg_data==[47])
seg_lcerebellumwm_labels, n_seg_lcerebellumwm_labels = label(seg_data==[7])
seg_rcerebellumwm_labels, n_seg_rcerebellumwm_labels = label(seg_data==[46])

#segmentation - lateral ventricle labels
seg_llv_labels, n_seg_llv_labels = label(seg_data==[4])
seg_rlv_labels, n_seg_rlv_labels = label(seg_data==[43])

#gm labels
gm_lh_labels, n_gm_lh_labels = label(gm_data==3)
gm_rh_labels, n_gm_rh_labels = label(gm_data==42)

## debugging tools for labels

In [364]:
print n_les_labels #prints number of lesion labels

print n_seg_brainstem_labels #prints number of labels in brainstem

print n_gm_lh_labels #prints number of labels in gray matter

print n_seg_llv_labels #prints number of labels in ventricle

print np.bincount(les_labels.ravel()) #prints volumes of lesions

print np.bincount(seg_lcerebellumwm_labels.ravel()) #prints volume of left cerebellum white matter

print np.bincount(gm_lh_labels.ravel()) #prints volume of left hemisphere gray matter

print np.bincount(seg_llv_labels.ravel()) #prints volume of left hemisphere ventricle

17
1
37
2
[9818227     326      77    4141      23      94      94      57      39
    5620     793      79     133     174     384      29      37      73]
[16766117    11099]
[16540965   236179        1        1        1        1        1       11
        1        1        1        3        1        1        1        1
        8        1        3        1        1        1        1        1
        1        1        1        1        1        1        1        1
       16        1        1        1        1        1]
[16771052     5845      319]


## Function that gets coordinates from labels

In [365]:
def get_label_coord(labels,num_labels):
    all_label_coords = []
    if num_labels >= 1:
        for count in range(1, num_labels+1):
            cur_label_coords = []
            x,y,z = np.nonzero(labels==count)
            for count2 in range(len(x)):
                cur_coord = [x[count2],y[count2],z[count2]]
                cur_label_coords.append(cur_coord)
            all_label_coords.append(cur_label_coords)
    else:
        x,y,z = np.nonzero(labels==1)
        for count in range(len(x)):
            all_label_coords.append([x[count],y[count],z[count]])
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_les_coords[count]
    #    print ""
    return all_label_coords

## Function that converts coordinates to real space

In [366]:
def get_rs_coord(coordinates,affine):
    all_rs_coords = []
    for count in range(len(coordinates)):
        cur_rs_coords = []
        for count2 in range(len(coordinates[count])):
            rs_coord = np.dot(affine, [coordinates[count][count2][0],
                                           coordinates[count][count2][1], 
                                           coordinates[count][count2][2],
                                           1])
            rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
            #rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
            #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
            cur_rs_coords.append(rs_coord_noone)
        all_rs_coords.append(cur_rs_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_rs_coords[count]
    #    print ""
    return all_rs_coords

## Function that converts coordinates to segmentation file coordinates

In [367]:
def get_seg_coord(coordinates):
    all_seg_coords = []
    for count in range(len(coordinates)):
        cur_seg_coords = []
        for count2 in range(len(coordinates[count])):
            seg_coord = np.dot(inv_seg_affine, [coordinates[count][count2][0],
                                                coordinates[count][count2][1], 
                                                coordinates[count][count2][2],
                                                1])
            #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
            #seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
            seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
            cur_seg_coords.append(seg_coord_noone)
        all_seg_coords.append(cur_seg_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_seg_coords[count]
    #    print ""
    return all_seg_coords

## Generate midbrain coordinates into a variable

In [368]:
bs = get_label_coord(seg_brainstem_labels,n_seg_brainstem_labels)
lcc = get_label_coord(seg_lcerebellumcortex_labels,n_seg_lcerebellumcortex_labels)
lcw = get_label_coord(seg_lcerebellumwm_labels,n_seg_lcerebellumwm_labels)
rcc = get_label_coord(seg_rcerebellumcortex_labels,n_seg_rcerebellumcortex_labels)
rcw = get_label_coord(seg_rcerebellumwm_labels,n_seg_rcerebellumwm_labels)

midbrain = []

if len(bs) != 1:
    midbrain.append(bs[1])
else:
    midbrain.append(bs[0])

if len(lcc) != 1:
    midbrain.append(lcc[1])
else:
    midbrain.append(lcc[0])

if len(lcw) != 1:
    midbrain.append(lcw[1])
else:
    midbrain.append(lcw[0])

if len(rcc) != 1:
    midbrain.append(rcc[1])
else:
    midbrain.append(rcc[0])

if len(rcw) != 1:
    midbrain.append(rcw[1])
else:
    midbrain.append(rcw[0])

print len(midbrain)



5


## Generate gray matter coordinates into a variable

In [369]:
gm = []
lhcoord = get_label_coord(gm_lh_labels,n_gm_lh_labels)
rhcoord = get_label_coord(gm_rh_labels,n_gm_rh_labels)
lhcoord_seg = get_seg_coord(get_rs_coord(lhcoord,gm_affine))
rhcoord_seg = get_seg_coord(get_rs_coord(rhcoord,gm_affine))

gm.append(lhcoord_seg[0])
gm.append(rhcoord_seg[0])

print len(gm[1])

239875


## Generate ventricular coordinates into a variable

In [370]:
ventricles = []
vlhcoord = get_label_coord(seg_llv_labels,n_seg_llv_labels)
vrhcoord = get_label_coord(seg_rlv_labels,n_seg_rlv_labels)

if len(vlhcoord) != 1:
    ventricles.append(vlhcoord[1])
else:
    ventricles.append(vlhcoord[0])
if len(vrhcoord) != 1:
    ventricles.append(vrhcoord[1])
else:
    ventricles.append(vrhcoord[0])

print len(ventricles)

2


## Generate lesion coordinates into a variable

In [371]:
lesions_les = get_label_coord(les_labels,n_les_labels)
lesions = get_seg_coord(get_rs_coord(lesions_les,les_affine))
lesions_seg = lesions

print len(lesions)

17


## Generate random coordinates from each structure

In [372]:
gm_test = []
mb_test = []
v_test = []
randcount = 10

def random_samp(lst,count):
    randlst = []
    for x in range(count):
        randlst.append(lst[random.randrange(0,len(lst))])
    return randlst

gm_lh_test = random_samp(gm[0],randcount)
gm_rh_test = random_samp(gm[1],randcount)
mb_bs_test = random_samp(midbrain[0],randcount)
mb_lcc_test = random_samp(midbrain[1],randcount)
mb_lcw_test = random_samp(midbrain[2],randcount)
mb_rcc_test = random_samp(midbrain[3],randcount)
mb_rcw_test = random_samp(midbrain[4],randcount)
v_lh_test = random_samp(ventricles[0],randcount)
v_rh_test = random_samp(ventricles[1],randcount)

for x in range(randcount):
    gm_test.append(gm_lh_test[x])
for x in range(randcount):
    gm_test.append(gm_rh_test[x])
for x in range(randcount):
    mb_test.append(mb_bs_test[x])
for x in range(randcount):
    mb_test.append(mb_lcc_test[x])
for x in range(randcount):
    mb_test.append(mb_lcw_test[x])
for x in range(randcount):
    mb_test.append(mb_rcc_test[x])
for x in range(randcount):
    mb_test.append(mb_rcw_test[x])
for x in range(randcount):
    v_test.append(v_lh_test[x])
for x in range(randcount):
    v_test.append(v_rh_test[x])
    
print gm_test
print mb_test
print v_test

[[155, 99, 155], [158, 172, 97], [173, 143, 71], [152, 114, 126], [159, 107, 57], [130, 129, 88], [129, 159, 44], [160, 97, 68], [166, 135, 134], [130, 85, 151], [93, 152, 45], [105, 142, 150], [71, 142, 100], [73, 138, 80], [97, 160, 115], [108, 154, 78], [104, 91, 117], [96, 115, 174], [120, 84, 89], [97, 170, 140]]
[[135, 171, 103], [122, 188, 100], [136, 166, 98], [138, 150, 101], [123, 167, 112], [135, 151, 103], [133, 201, 100], [128, 193, 106], [137, 180, 97], [122, 184, 100], [169, 187, 66], [149, 173, 105], [132, 191, 95], [137, 186, 85], [138, 198, 82], [152, 188, 79], [150, 179, 81], [141, 175, 82], [136, 186, 68], [141, 182, 68], [152, 180, 96], [144, 173, 94], [146, 176, 94], [145, 188, 99], [151, 185, 97], [152, 195, 82], [147, 184, 96], [145, 178, 104], [149, 178, 98], [153, 194, 82], [95, 184, 75], [86, 190, 87], [97, 198, 79], [124, 188, 92], [117, 186, 59], [111, 198, 61], [92, 204, 79], [124, 173, 73], [114, 198, 56], [104, 189, 63], [121, 181, 85], [124, 176, 82], [

## Convert coordinates from nested lists to strings

In [373]:
def convert_coord_str(coordinates):
    coord_str = []
    for x in range(len(coordinates)):
        str_int = []
        for y in range(len(coordinates[x])):
            str_int.append(str(coordinates[x][y]))
        coord_str.append(str_int)
    return coord_str

lesions_str = convert_coord_str(lesions)
midbrain_str = convert_coord_str(midbrain)

## Averaging function

In [389]:
def average_func(coordinates):
    sumx=0;sumy=0;sumz=0
    for count in range(len(coordinates)):
        sumx += coordinates[count][0]
        sumy += coordinates[count][1]
        sumz += coordinates[count][2]
    average_x = sumx / len(coordinates)
    average_y = sumy / len(coordinates)
    average_z = sumz / len(coordinates)
    #print count, averages
    return [average_x, average_y, average_z]

## Take average of all lesion coordinates

In [390]:
les_averages = []
for x in range(len(lesions)):
    les_averages.append(average_func(lesions[x]))
print les_averages

[[96, 145, 81], [95, 154, 102], [106, 118, 102], [104, 109, 128], [109, 181, 94], [108, 128, 96], [111, 117, 134], [118, 177, 109], [145, 117, 109], [146, 133, 70], [145, 120, 150], [151, 128, 158], [156, 136, 76], [160, 152, 85], [159, 186, 90], [163, 159, 134], [164, 157, 123]]


## Infratentorial Check V2 

In [374]:
les_type = []
count = 0
while count <= (len(lesions_str))-1:
    lesion_type = ""
    x = set(lesions_str[count])
    count2 = 0
    while count2 <= (len(midbrain_str))-1:
        y = set(midbrain_str[count2])
        testint = x.isdisjoint(y)
        if testint == False:
            count2 = len(midbrain_str)
        count2 += 1
    if testint == False:
        lesion_type = "infratentorial"
    else:
        lesion_type = "subcortical"
    print count+1, lesion_type
    les_type.append(lesion_type)
    count += 1

1 subcortical
2 subcortical
3 subcortical
4 subcortical
5 infratentorial
6 subcortical
7 subcortical
8 infratentorial
9 subcortical
10 subcortical
11 subcortical
12 subcortical
13 subcortical
14 subcortical
15 infratentorial
16 subcortical
17 subcortical


## append results to a new list 

In [375]:
results = []
sub_count, inf_count, jux_count, per_count, err_count = 0, 0, 0, 0, 0
for count in range(len(les_type)):
    results.append([count+1, les_type[count]])
    if les_type[count] == "subcortical":
        sub_count += 1
    elif les_type[count] == "infratentorial":
        inf_count += 1
    elif les_type[count] == "juxtacortical":
        jux_count += 1
    elif les_type[count] == "periventricular":
        per_count += 1
    else:
        err_count += 1
all_sub_results.append([subjects[0], 
                        results, 
                        sub_count, 
                        jux_count,
                        per_count,
                        inf_count, 
                        err_count]
                      )
print all_sub_results

[['mseID', 'lesion number and type', 'number of subcortical lesions', 'number of juxtacortical lesions', 'number of periventricular lesions', 'number of infratentorial lesions', 'number of errors'], ['mse1474', [[1, 'subcortical'], [2, 'subcortical'], [3, 'subcortical'], [4, 'subcortical'], [5, 'infratentorial'], [6, 'subcortical'], [7, 'subcortical'], [8, 'infratentorial'], [9, 'subcortical'], [10, 'subcortical'], [11, 'subcortical'], [12, 'subcortical'], [13, 'subcortical'], [14, 'subcortical'], [15, 'infratentorial'], [16, 'subcortical'], [17, 'subcortical']], 14, 0, 0, 3, 0]]


In [376]:
print lesions[4][4]
print lesions[7][4]
print lesions[14][8]

[107, 181, 94]
[117, 178, 111]
[159, 187, 91]


# The debugging portions of the code

## Infratentorial Lesion Check

les_type = []
for count in range(len(lesions)):
    lesion_type = ""
    count4 = 0
    if len(lesions[count]) <= 500:
        for count2 in range(len(lesions[count])):
            halt = ""
            for count3 in range(len(midbrain)):
                for count4 in range(len(midbrain[count3])):
                    if lesions[count][count2] == midbrain[count3][count4]:
                        lesion_type = "infratentorial"
                        halt = "stop"
                    if halt == "stop":
                        continue
                if halt == "stop":
                    continue
            if halt == "stop":
                continue
        if lesion_type == "":
            lesion_type = "subcortical"
        print count, "done, type is:", lesion_type
        les_type.append(lesion_type)
    else:
        print "skipped lesion number", count, "; too large"
        les_type.append("too large")

print les_type

## Debugging function that finds range of values in brain lesion/segment

In [377]:
test_x = [];test_y=[];test_z=[]
for count in range(len(midbrain[0])):
    test_x.append(midbrain[0][count][0])
    test_y.append(midbrain[0][count][1])
    test_z.append(midbrain[0][count][2])

def range_coord(val_list):
    min_val = min(val_list)
    max_val = max(val_list)
    print min_val,"-", max_val

range_coord(test_x);range_coord(test_y);range_coord(test_z)

114 - 145
149 - 208
92 - 123


## Debugging Function that searches for coordinates in any given brain lesion/segment

In [378]:
def search(coordinates,check_coord):
    number = 0
    for count in range(len(coordinates)):
        if check_coord == coordinates[count]:
            number = count + 1
            print count
    if number == 0:
        print "not available"
    else:
        print "available"

#search(midbrain[0],lesions[7][39])
#search(midbrain[0],[144, 177, 100])
#search(lesions[7],lesions[7][39])

## Debugging Coding Section

In [ ]:
realspace_coordinates = np.dot(les_affine, [average_les[0][0],
                 average_les[0][1], 
                 average_les[0][2],
                 1])
average_les[0] 
realspace_coordinates
from numpy.linalg import inv
inv_seg_affine = np.linalg.inv(seg_affine)
freesurfer_matrix = np.dot(inv_seg_affine, [realspace_coordinates[0],
                                            realspace_coordinates[1], 
                                            realspace_coordinates[2],
                                            1])
print average_les[0]; print realspace_coordinates; print freesurfer_matrix

print len(midbrain[0])

les_type = []

"""for test_count in range(len(lesions)):
    for count in range(len(lesions[test_count])):
        for count2 in range(len(midbrain)):
            halt = ""
            for count3 in range(len(midbrain[count2])):
                if lesions[test_count][count] == midbrain[count2][count3]:
                    inf_num += 1
                    lesion_type = "infratentorial"
                    halt = "stop"
                    break
            if halt == "stop":
                break
    print lesion_type
if lesion_type == "infratentorial":
    les_type.append(lesion_type)
elif lesion_type == "":
    les_type.append("subcortical")
print les_type
print inf_num
"""

average_les = average_func(get_label_coord(les_labels, n_les_labels))
average_rs = average_func(get_rs_coord(get_label_coord(les_labels, n_les_labels)))
average_seg = average_func(get_seg_coord(get_rs_coord(get_label_coord(les_labels, n_les_labels))))

allcoord_les = get_label_coord(les_labels, n_les_labels)
allcoord_rs = get_rs_coord(get_label_coord(les_labels, n_les_labels))
allcoord_seg = get_seg_coord(get_rs_coord(get_label_coord(les_labels, n_les_labels)))

print average_les[0]
print average_rs[0]
print average_seg[0]
print allcoord_les[0][0]
print allcoord_rs[0][0]
print allcoord_seg[0][0]

print average_seg

print lesions[8][20]